## Quiz #0801

### "Text Classification with Keras"

In [1]:
import numpy as np
import pandas as pd
import re
import nltk
import os
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from sklearn.datasets import load_files
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, LSTM, Embedding, SpatialDropout1D
from tensorflow.keras.utils import to_categorical
from keras.preprocessing import sequence
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences 
#nltk.download('stopwords')

#### Answer the following question by providing Python code:

1). Read in the movie review data from Cornell CS department. Carry out the EDA. <br>
- The data can be found [here](https://www.cs.cornell.edu/people/pabo/movie-review-data). <br>
- Download the “polarity dataset” and unzip. <br>
- Under the "txt_sentoken” folder, there are “pos” and “neg" subfolders. <br>

In [2]:
reviews = load_files('txt_sentoken/')
my_docs, y = reviews.data, reviews.target

In [3]:
print(len(my_docs))
print(len(y))


2000
2000


2). Carry out the data preprocessing: <br>
- Cleaning.
- Stopword removal.

In [4]:
#clean
def cleantxt(txt):
    
    stpw = stopwords.words('english')  
      
    txt=txt.decode('utf-8')
    txt = re.sub(r"\n", " ", txt)
    txt = re.sub("[\<\[].*?[\>\]]", " ", txt)
    txt = txt.lower()
    txt = re.sub(r"[^a-z ]", " ", txt)
    txt = re.sub(r"\b\w{1,3}\b", " ",txt)
    txt = " ".join([x for x in txt.split() if x not in stpw])
    
    return txt

my_docs = list(map(cleantxt, my_docs))



3). Carry out label encoding by integers (required form by Keras):

In [55]:
MAX_NB_WORDS = 50000
#tokenizer
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(my_docs)
print('unique tokens : ' ,len(tokenizer.word_index))

#sequencing
MAX_SEQUENCE_LENGTH = 250
X = tokenizer.texts_to_sequences(my_docs)

#padding
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)

unique tokens :  37556


4). Prepare the data for AI: <br>
- Apply the padding.
- Split the data into training and testing.

In [6]:
#split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)


5). Define the AI model (Embedding + LSTM):

In [48]:
embed_dim = 100

model = Sequential(
    [
        Embedding(MAX_NB_WORDS, embed_dim, input_length=X_train.shape[1]),
        SpatialDropout1D(0.15),
        LSTM(100, dropout=0.2, recurrent_dropout=0.2),
        Dense(2, activation='sigmoid')
    ]
)




6). Define the optimizer and compile the model:

In [49]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

7). Train the model and visualize the summary:

In [50]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 250, 100)          5000000   
_________________________________________________________________
spatial_dropout1d_9 (Spatial (None, 250, 100)          0         
_________________________________________________________________
lstm_9 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_9 (Dense)              (None, 2)                 202       
Total params: 5,080,602
Trainable params: 5,080,602
Non-trainable params: 0
_________________________________________________________________


In [51]:

epochs = 6
batch_size = 64

history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1)

Epoch 1/6
23/23 [==============================] - 25s 896ms/step - loss: 0.6917 - accuracy: 0.5111 - val_loss: 0.6846 - val_accuracy: 0.5312
Epoch 2/6
23/23 [==============================] - 22s 932ms/step - loss: 0.5756 - accuracy: 0.7118 - val_loss: 0.5475 - val_accuracy: 0.7688
Epoch 3/6
23/23 [==============================] - 31s 1s/step - loss: 0.2195 - accuracy: 0.9458 - val_loss: 0.4425 - val_accuracy: 0.8125
Epoch 4/6
23/23 [==============================] - 27s 1s/step - loss: 0.0387 - accuracy: 0.9903 - val_loss: 0.5394 - val_accuracy: 0.7812
Epoch 5/6
23/23 [==============================] - 34s 1s/step - loss: 0.0113 - accuracy: 0.9993 - val_loss: 0.7626 - val_accuracy: 0.7688
Epoch 6/6
23/23 [==============================] - 25s 1s/step - loss: 0.0029 - accuracy: 1.0000 - val_loss: 0.6181 - val_accuracy: 0.8687


8). Display the test result (accuracy):

In [53]:
score = model.evaluate(X_test,y_test)
print('Accuracy: ',score[1] )

13/13 [==============================] - 1s 114ms/step - loss: 0.5073 - accuracy: 0.8525
Accuracy:  0.8525000214576721
